导入了d2l包中的绘图类，方便直观地绘制损失的变换

In [ ]:
from __future__ import print_function
!pip install d2l==0.17.5.
%matplotlib inline
from d2l import torch as d2l

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models

import os
from google.colab import drive

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

自定义图片加载方法，将图片转换到需要的尺寸（128 or 512）和格式（torch tensor）

In [ ]:
imsize = 512 if torch.cuda.is_available() else 128  # 如果没有GPU，就使用较小的图像尺寸

loader = transforms.Compose([
    transforms.Resize(imsize),  # 修改图片尺寸
    transforms.ToTensor()])  # 转成torch tensor格式


def image_loader(image_name):
    image = Image.open(image_name)
    # 拟合网络输入尺寸所需的额外批维度
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


从Drive云端硬盘读取素材图片

In [ ]:
# 从云端硬盘读取输入
drive.mount('/content/drive')
path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

In [ ]:
# 内容图像和风格图像
content_img = image_loader("xmlg.jpg")
style_img = image_loader("bjs.jpg")

转换成PIL格式便于后续操作

In [ ]:
plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # 在副本上操作
    image = image.squeeze(0)      # 移除此前设置的额外维度
    image = transforms.ToPILImage()(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.01) # 暂停一会等待绘图完成

# plt.figure()
imshow(content_img, title='Content Image')

# plt.figure()
imshow(style_img, title='Style Image')

定义两个距离，一个用于内容Dc和一个用于样式 Ds。Dc测量两个图像之间的内容差异，同时Ds测量两个图像之间的样式差异。然后，获取第三个图像，即输入，并对其进行转换，以最小化其与内容图像的内容距离和与样式图像的样式距离。

内容损失函数表示加权后单层内容与原输入间的距离，该函数采用特征图的图层在网络处理输入中X并返回加权内容和之间距离。内容图像的特征图。将此函数实现为torch模块，其构造函数作为输入。距||FXL−FCL||2是两组特征映射之间的均方误差，可以使用nn.MSELoss 计算。

In [ ]:
class ContentLoss(nn.Module): # 内容损失

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.loss = None
        self.target = target.detach()

    def forward(self, input):
        self.loss = func.mse_loss(input, self.target)
        return input

样式丢失模块的实现方式与内容丢失模块类似。它将充当网络中的透明层，用于计算该层的样式损失。为了计算样式损失，需要计算gram矩阵GXL.格拉姆矩阵是将给定矩阵乘以其转置矩阵的结果。

必须通过将每个元素除以矩阵中的元素总数来规范化格拉姆矩阵。这种规范化是为了抵消以下影响： F^XL具有很大的矩阵维度N，在 Gram 矩阵中产生较大的值。这些较大的值将导致第一层（在池化层之前）在梯度下降期间产生更大的影响，而样式特征往往位于网络的更深层。

In [ ]:
def gram_matrix(input): # 格拉姆矩阵
    channels, n = input.shape[1], input.numel() // input.shape[1]
    input = input.reshape((channels, n))
    return torch.matmul(input, input.T) / (channels * n)

In [ ]:
class StyleLoss(nn.Module): #风格损失

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.loss = None
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        g = gram_matrix(input)
        self.loss = func.mse_loss(g, self.target)
        return input

PyTorch的VGG实现是一个模块，分为两个Sequential 子模块：features （卷积层和池化层）和classifier （完全连接层）。这里使用features 模块，因为需要各个卷积层的输出来测量内容和样式损失。某些层在训练期间的行为与评估时的行为不同，因此必须使用.eval()将网络设置为评估模式。VGG网络在图像上进行训练，每个通道归一化为meanst=[0.485， 0.456， 0.406]和std=[0.229， 0.224， 0.225]。在将图像输入到网络之前，将使用它们对其进行规范化。

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [ ]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module): # 标准化以便在网络上操作
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # 使其变成 [C x 1 x 1] 以便于直接和 [B x C x H x W] 的图像张量操作
        # B(batch size)批处理大小. C(channels) 通道数. H(height) 高. W(width) 宽.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # 标准化图像张量
        return (img - self.mean) / self.std

使用L-BFGS算法来运行梯度下降。与训练网络不同，这里希望训练输入图像，以尽量减少内容/样式损失。将创建一个PyTorch L-BFGS优化器optim.LBFGS，并将图像作为要优化的张量传递给它。

In [ ]:
# 计算 内容/风格 损失 :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    # 标准化模块
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # 损失
    content_losses = []
    style_losses = []

    # 假定cnn是nn.Sequential
    model = nn.Sequential(normalization)

    i = 0  # 每次遇到卷积层时+1
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # 原地操作不适用于内容损失函数和风格损失函数，所以这里采用异地操作
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 最后一次计算完内容损失和风格损失后裁剪图层
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

In [ ]:
input_img = content_img.clone()

plt.figure()
imshow(input_img, title='Input Image')

使用L-BFGS算法来运行梯度下降。与训练网络不同，这里希望训练输入图像，以尽量减少内容/样式损失。将创建一个PyTorch L-BFGS优化器optim.LBFGS，并将图像作为要优化的张量传递给它。

In [ ]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=10000, content_weight=1):

    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[50, num_steps],
                            legend=['content', 'style','TV'],
                            ncols=1, figsize=(7, 2.5))

    """开始运行风格迁移."""
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)

    # 需要优化输入的拷贝而不是模型的参数，所以根据需要更新所有的梯度
    input_img.requires_grad_(True)
    model.requires_grad_(False)

    optimizer = optim.LBFGS([input_img])

    run = [0]
    while run[0] <= num_steps:

        def closure():
            # 更正更新后的输入
            with torch.no_grad():
                input_img.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1

            if (run[0]+1) % 10 == 0 and run[0]>=49:
                animator.add(run[0], [float(content_score.item()),
                                     float(style_score.item()),float(loss.item())])

            return style_score + content_score

        optimizer.step(closure)

    # 最后还需修正
    with torch.no_grad():
        input_img.clamp_(0, 1)

    return input_img

In [ ]:
output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img)
plt.figure()
imshow(output, title='Output Image')

plt.ioff()
plt.show()